In [1]:
#モジュールを自動リロード
%load_ext autoreload
%autoreload 2

from LLMSearch.RWKVEmbedding import RWKVEmbedding
from LLMSearch.VicunaEmbedding import VicunaEmbedding

In [2]:
import json
with open('settings/settings.json') as f:
    settings = json.load(f)


In [3]:
#embedder=RWKVEmbedding(settings)
embedder=VicunaEmbedding()

In [4]:
from LLMSearch.TextSplitter import auto_txt_split

glob_path=settings["data_path"] + "/texts/*.txt"
chunk_size_limit= settings["chunk_size_limit"]
auto_txt_split(glob_path, chunk_size_limit,init=True)

UserData/split/ginga_tets.txt_chunk_0.txt
UserData/split/ginga_tets.txt_chunk_1.txt
UserData/split/wagahai.txt_chunk_0.txt


[nltk_data] Downloading package punkt to /home/kh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#chunk_path_listからそれぞれデータを読み込み､RWKVによる埋め込みを行う
import numpy as np
from LLMSearch.TextSearcher import TextSearcher
from tqdm import tqdm
import glob


chunk_path_list=glob.glob(settings["data_path"] + "/split/*.txt")
searcher=TextSearcher(embedder,settings["data_path"])

#searcher.embedder.mean_mode=False

for path in tqdm(chunk_path_list):
    searcher.calc_text_file(path)

100%|██████████| 3/3 [00:10<00:00,  3.39s/it]


In [6]:
searcher.save_model()
searcher.load_model()

In [7]:
txt="I am a cat "
txt="猫である"
#txt="吾輩は猫である "*10
#txt="吾輩は猫である。名前はまだ無い。"
#txt="あなたはわかっているでしょう"
k=3

searcher.search(txt,k)

[{'path': 'UserData/split/ginga_tets.txt_chunk_1.txt',
  'text': '「ジョバンニさん。あなたはわかっているのでしょう。」',
  'sim': 0.98184556},
 {'path': 'UserData/split/ginga_tets.txt_chunk_0.txt',
  'text': '「ではみなさんは、そういうふうに川だと云われたり、乳の流れたあとだと云われたりしていたこのぼんやりと白いものがほんとうは何かご承知ですか。」先生は、黒板に吊した大きな黒い星座の図の、上から下へ白くけぶった銀河帯のようなところを指しながら、みんなに問をかけました。 カムパネルラが手をあげました。それから四五人手をあげました。ジョバンニも手をあげようとして、急いでそのままやめました。たしかにあれがみんな星だと、いつか雑誌で読んだのでしたが、このごろはジョバンニはまるで毎日教室でもねむく、本を読むひまも読む本もないので、なんだかどんなこともよくわからないという気持ちがするのでした。 ところが先生は早くもそれを見附けたのでした。',
  'sim': 0.87590176},
 {'path': 'UserData/split/wagahai.txt_chunk_0.txt',
  'text': '吾輩は猫である。名前はまだ無い。 どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。',
  'sim': 0.86525106}]